In [2]:
import pandas as pd
from tqdm import tqdm
import numpy as np

import jieba
import re

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader,Dataset
from sklearn.model_selection import train_test_split

from transformers import BertTokenizer
from gensim.models.word2vec import Word2Vec
import math

device = 'cuda:0'

c:\Users\15835\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
#提取源数据
raw_data = pd.read_csv('train.csv')
raw_data.head()

,id,fact,label
0,id_0,罪犯陈中盛，男，1954年xx月xx日出生，汉族，湖南省永兴县人，文盲，现在河南省新郑监狱服...,12
1,id_1,罪犯王中国，男，1982年xx月xx日生，汉族，吉林省长春市宽城区人，初中毕业，现在吉林省长...,13
2,id_2,罪犯张仁奇，现在宁夏回族自治区银川监狱服刑，以被告人张仁奇犯信用卡诈骗罪，判处有期徒刑刑期六...,8
3,id_3,罪犯吴晨，男，1988年xx月xx日出生，瑶族，广西巴马县人，初中文化，原住广西巴马县巴马镇...,7
4,id_4,罪犯夏宏，女，1980年xx月xx日出生，汉族，初中文化，湖南省邵阳市大祥区人，住湖南省邵阳...,6


In [3]:
#清洗数据


def text_preprocess(text):
    text = re.sub(r'<.*?>', '', text)#HTML标签
    text = re.sub(r'[^\w\s]', '', text)#特殊符号
    text = text.replace(' ', '')#空格

    with open('hit_stopwords.txt', 'r', encoding='utf-8') as f:
        stopwords = [line.strip() for line in f.readlines()]
    text = jieba.cut(text)
    text = ' '.join([word for word in text if word not in stopwords])
    pattern = re.compile(r'([\u4e00-\u9fa5]{2,5}?(?:省|自治区|市)){0,1}([\u4e00-\u9fa5]{2,7}?(?:区|县|州)){0,1}([\u4e00-\u9fa5]{2,7}?(?:镇)){0,1}([\u4e00-\u9fa5]{2,7}?(?:村|街|街道)){0,1}([\d]{1,3}?(号)){0,1}')
    text =pattern.sub( '', text)
    return text


In [4]:
clean_list = [text_preprocess(text) for text in raw_data['fact'].to_list()]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\15835\AppData\Local\Temp\jieba.cache
Loading model cost 0.431 seconds.
Prefix dict has been built successfully.


In [5]:
raw_data['clean_fact'] = clean_list
raw_data['fact_len'] = [len(i.split()) for i in clean_list]
raw_data['fact_len'].describe(percentiles=[.5,.95])
raw_data.to_csv('processed_data.csv')
X_train, X_test, y_train, y_test = train_test_split(raw_data.clean_fact, raw_data.label, test_size=0.2, random_state=42, shuffle=False)

In [ ]:
w2v = Word2Vec(vector_size=400,
               min_count=10)    
 
w2v.build_vocab(X_train)
w2v.train(X_train,total_examples=w2v.corpus_count,epochs=40)
w2v.save('./word2vec/word2vec.model')

In [8]:
raw_data = pd.read_csv('processed_data.csv')
X_train, X_test, y_train, y_test = train_test_split(raw_data.clean_fact, raw_data.label.values, test_size=0.2, random_state=42, shuffle=False)
len(y_test)

10001

In [3]:


X_train, X_test, y_train, y_test = train_test_split(raw_data.clean_fact, raw_data.label.values, test_size=0.2, random_state=42, shuffle=False)

from gensim.models.word2vec import Word2Vec
def average_vec(text):
    vec = np.zeros(400).reshape((1, 400))
    for word in text:
        try:
            vec += w2v.wv[word].reshape((1, 400))
        except KeyError:
            continue
    return vec

w2v = Word2Vec.load("word2vec\word2vec.model")
X_train_vec = np.concatenate([average_vec(z) for z in X_train])
X_test_vec = np.concatenate([average_vec(z) for z in X_test])


In [17]:
class myDataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.labels = label
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return torch.tensor(sample, dtype=torch.float32).unsqueeze(0), torch.tensor(label, dtype=torch.float)
    def __len__(self):
        return len(self.data)

In [152]:
class W2V_CNN(nn.Module):
    def __init__(self, input_dim, output_dim, conv_kernel_size, maxpool_kernel_size, stride, padding, linear_input):
        super(W2V_CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels = input_dim, out_channels = 16, kernel_size = conv_kernel_size, stride = stride, padding = padding)
        self.conv2 = nn.Conv1d(in_channels = 16, out_channels = 32, kernel_size = conv_kernel_size, stride = stride, padding = padding)
        self.max_pool = nn.MaxPool1d(kernel_size = maxpool_kernel_size) 
        self.l1 = nn.Linear(linear_input, 1024)
        self.l2 = nn.Linear(1024,256)
        self.l3 = nn.Linear(256,output_dim)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(0.3)

    def forward(self, x):
        ####### Your code between this #########
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool(x)

        x = self.conv2(x)
        x = self.max_pool(x)
        x = self.relu(x)
        
        #print (x.size())
        x = x.view(-1,32*100)
        x = self.l1(self.relu(x))
        x = self.drop(x)
        x = self.l2(self.relu(x))
        x = self.drop(x)
        x = self.l3(self.relu(x))
        x = F.log_softmax(x,dim=1)
        #self.out(x)
        return x


In [214]:
torch.cuda.empty_cache()

input_dim = 1
output_dim = 34
conv_kernel_size = 3
maxpool_kernel_size = 2
stride = 1
padding = 1
linear_input = 32*100

LR = 1e-3
EPOCHS = 30

train_dataset = myDataset(X_train_vec, y_train)
train_loader = DataLoader(train_dataset, batch_size=9192, shuffle=True)
model = W2V_CNN(input_dim, output_dim, conv_kernel_size, maxpool_kernel_size, stride, padding, linear_input).to(device)


criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=LR)

In [215]:
for epoch in range(EPOCHS):
    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        ####### Your code between this #########
        output = model(inputs)
        optimizer.zero_grad()
        loss = criterion(output, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        ####### Your code between this #########
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{EPOCHS}], Loss: {running_loss:.4f}')
print("Traning finished !")


Epoch [1/30], Loss: 299.1339
Epoch [2/30], Loss: 65.4407
Epoch [3/30], Loss: 17.1878
Epoch [4/30], Loss: 15.7820
Epoch [5/30], Loss: 14.8887
Epoch [6/30], Loss: 14.3976
Epoch [7/30], Loss: 14.1127
Epoch [8/30], Loss: 13.9519
Epoch [9/30], Loss: 13.7249
Epoch [10/30], Loss: 13.5634
Epoch [11/30], Loss: 13.3652
Epoch [12/30], Loss: 13.2359
Epoch [13/30], Loss: 13.0493
Epoch [14/30], Loss: 12.9706
Epoch [15/30], Loss: 12.8262
Epoch [16/30], Loss: 12.7004
Epoch [17/30], Loss: 12.6004
Epoch [18/30], Loss: 12.4972
Epoch [19/30], Loss: 12.4179
Epoch [20/30], Loss: 12.3905
Epoch [21/30], Loss: 12.2804
Epoch [22/30], Loss: 12.2515
Epoch [23/30], Loss: 12.1636
Epoch [24/30], Loss: 12.1020
Epoch [25/30], Loss: 12.0269
Epoch [26/30], Loss: 11.9755
Epoch [27/30], Loss: 11.9315
Epoch [28/30], Loss: 11.8673
Epoch [29/30], Loss: 11.8438
Epoch [30/30], Loss: 11.7743
Traning finished !


In [157]:
torch.save(model,'./models/W2V_CNN.model')

In [211]:
torch.cuda.empty_cache()
model = torch.load('./models/W2V_CNN.model').to(device)

In [4]:
def Score1(pred,true):
    score = 0
    v = torch.abs(torch.log(pred+1)-torch.log(true+1))
    for i in v:
        if i.item()<=0.2:
            score += 1
        elif i.item() <= 0.4:
            score+=0.8
        elif i.item() <=0.6:
            score+=0.6
        elif i.item() <=0.8:
            score += 0.4
        elif i.item() <=1:
            score += 0.2
    return score
    

In [218]:



test_dataset = myDataset(X_test_vec,y_test.to_list())
test_loader = DataLoader(test_dataset,shuffle=True,batch_size=200)


correct = 0
total = 0
score1 = 0
model.eval()
with torch.no_grad():
    for i in test_loader:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(torch.exp(outputs.data), 1)
        total += labels.size(0)
        score1 += Score1(predicted,labels)
        correct += (predicted == labels).sum().item()   

ExtAcc = correct/total

print(f"ExtAcc:{ExtAcc},Score1:{score1},FinalScore:{0.3*ExtAcc+0.7*score1/total}")

ExtAcc:0.24350247524752475,Score1:132090.00000000003,FinalScore:0.6340037128712872


In [3]:

def preprocess_data(df, tokenizer, max_length):
    inputs = tokenizer(
        df['fact'].tolist(),
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    return inputs

# 使用BERT tokenizer进行文本编码
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
train_inputs = preprocess_data(raw_data, tokenizer, max_length=512)


In [4]:
class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_size, num_classes, kernel_sizes, num_channels):
        super(TextCNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_channels, (k, embed_size)) for k in kernel_sizes
        ])
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(len(kernel_sizes) * num_channels, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(1)  # (batch_size, 1, seq_len, embed_size)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs]  # [(batch_size, num_channels, seq_len), ...]
        x = [F.max_pool1d(item, item.size(2)).squeeze(2) for item in x]  # [(batch_size, num_channels), ...]
        x = torch.cat(x, 1)
        x = self.dropout(x)
        logits = self.fc(x)
        return F.log_softmax(logits,dim=1)

In [4]:
train_labels = raw_data['label'].values

X_train, X_val, attention_mask_train, attention_mask_val, y_train, y_val = train_test_split(
    train_inputs['input_ids'], train_inputs['attention_mask'], train_labels, test_size=0.2, random_state=42
)

class CustomDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item
    
train_inputs_train_dict = {'input_ids': X_train, 'attention_mask': attention_mask_train}
train_inputs_val_dict = {'input_ids': X_val, 'attention_mask': attention_mask_val}

train_dataset = CustomDataset(train_inputs_train_dict, y_train)
val_dataset = CustomDataset(train_inputs_val_dict, y_val)

train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)




In [6]:


torch.cuda.empty_cache()
vocab_size = len(tokenizer.vocab)
embed_size = 512
num_classes = 34
kernel_sizes = [3, 4, 5]
num_channels = 100

model = TextCNN(vocab_size, embed_size, num_classes, kernel_sizes, num_channels)

model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

In [7]:

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_predictions = []
    train_labels_list = []
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_predictions.extend(outputs.cpu().detach().numpy())
        train_labels_list.extend(labels.cpu().detach().numpy())
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}')

Training Epoch 1/10: 100%|██████████| 40/40 [00:22<00:00,  1.78it/s]


Epoch [1/10], Loss: 147.7819


Training Epoch 2/10: 100%|██████████| 40/40 [00:22<00:00,  1.81it/s]


Epoch [2/10], Loss: 132.5046


Training Epoch 3/10: 100%|██████████| 40/40 [00:22<00:00,  1.81it/s]


...

Epoch [8/10], Loss: 118.1221


Training Epoch 9/10: 100%|██████████| 40/40 [00:22<00:00,  1.76it/s]


Epoch [9/10], Loss: 116.7963


Training Epoch 10/10: 100%|██████████| 40/40 [00:22<00:00,  1.81it/s]

Epoch [10/10], Loss: 115.4546


In [253]:
torch.save(model,'./models/TextCNN.model')

In [10]:
correct = 0
total = 0
score1 = 0
model.eval()
with torch.no_grad():
    for batch in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}"):
        inputs = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(torch.exp(outputs.data), 1)
        total += labels.size(0)
        score1 += Score1(predicted,labels)
        correct += (predicted == labels).sum().item()   

ExtAcc = correct/total

print(f"ExtAcc:{ExtAcc},Score1:{score1},FinalScore:{0.3*ExtAcc+0.7*score1/total}")

Validation Epoch 10/10: 100%|██████████| 10/10 [00:02<00:00,  3.57it/s]

ExtAcc:0.23097690230976903,Score1:7673.599999999989,FinalScore:0.6063913608639128


In [5]:
class TextRNN_Att(nn.Module):
    def __init__(self, vocab_size, embed_size,hidden_size,num_layers,hidden_size2,num_classes):
        super(TextRNN_Att, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers,
                            bidirectional=True, batch_first=True, dropout=0.5)
        self.tanh1 = nn.Tanh()
        # self.u = nn.Parameter(torch.Tensor(config.hidden_size * 2, config.hidden_size * 2))
        self.w = nn.Parameter(torch.zeros(hidden_size * 2))
        self.tanh2 = nn.Tanh()
        self.fc1 = nn.Linear(hidden_size * 2, hidden_size2)
        self.fc = nn.Linear(hidden_size2, num_classes)

    def forward(self, x):
        x = x
        emb = self.embedding(x)  # [batch_size, seq_len, embeding]=[128, 32, 300]
        H, _ = self.lstm(emb)  # [batch_size, seq_len, hidden_size * num_direction]=[128, 32, 256]

        M = self.tanh1(H)  # [128, 32, 256]
        # M = torch.tanh(torch.matmul(H, self.u))
        alpha = F.softmax(torch.matmul(M, self.w), dim=1).unsqueeze(-1)  # [128, 32, 1]
        out = H * alpha  # [128, 32, 256]
        out = torch.sum(out, 1)  # [128, 256]
        out = F.relu(out)
        out = self.fc1(out)
        out = self.fc(out)  # [128, 64]
        return out

In [6]:
torch.cuda.empty_cache()
vocab_size = len(tokenizer.vocab)
embed_size = 512
num_classes = 34
hidden_size = 128                                       
num_layers = 2                                            
hidden_size2 = 64

model = TextRNN_Att(vocab_size, embed_size,hidden_size,num_layers,hidden_size2,num_classes=num_classes)

model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adagrad(params=model.parameters(), lr=0.01)

In [7]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_predictions = []
    train_labels_list = []
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        input_ids = input_ids.to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_predictions.extend(outputs.cpu().detach().numpy())
        train_labels_list.extend(labels.cpu().detach().numpy())
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}')

Training Epoch 1/10: 100%|██████████| 40/40 [03:40<00:00,  5.51s/it]


Epoch [1/10], Loss: 113.5696


Training Epoch 2/10: 100%|██████████| 40/40 [03:37<00:00,  5.43s/it]


Epoch [2/10], Loss: 104.8152


Training Epoch 3/10: 100%|██████████| 40/40 [03:44<00:00,  5.61s/it]


...

Epoch [8/10], Loss: 84.0526


Training Epoch 9/10: 100%|██████████| 40/40 [05:02<00:00,  7.56s/it]


Epoch [9/10], Loss: 81.9910


Training Epoch 10/10: 100%|██████████| 40/40 [05:12<00:00,  7.82s/it]

Epoch [10/10], Loss: 79.8581


In [14]:
correct = 0
total = 0
score1 = 0
model.eval()
with torch.no_grad():
    for batch in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}"):
        inputs = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(torch.exp(outputs.data), 1)
        total += labels.size(0)
        score1 += Score1(predicted,labels)
        correct += (predicted == labels).sum().item()   

ExtAcc = correct/total

print(f"ExtAcc:{ExtAcc},Score1:{score1},FinalScore:{0.3*ExtAcc+0.7*score1/total}")

Validation Epoch 10/10: 100%|██████████| 10/10 [00:15<00:00,  1.55s/it]

ExtAcc:0.3845615438456154,Score1:8870.799999999985,FinalScore:0.7362623737626226


In [14]:
torch.save(model,"./models/TxtxRNN_Att.model")

In [11]:
testA = pd.read_csv('testA.csv')
test_inputs = preprocess_data(testA, tokenizer, max_length=512)
test_inputs_train_dict = {'input_ids': test_inputs['input_ids'], 'attention_mask': test_inputs['attention_mask']}
labels = [0]*len(testA)

dataset = CustomDataset(test_inputs_train_dict,labels)
test_loader_A = DataLoader(dataset, batch_size=8, shuffle=False)

def predict(model, dataloader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Predicting"):
            input_ids = batch['input_ids'].to(device)
            outputs = model(input_ids)
            _,predict= torch.max(torch.exp(outputs.data), 1)
            predictions.extend(predict.cpu().numpy())
    return predictions

predictions_A = predict(model, test_loader_A)
submission =pd.DataFrame()
submission['id']=testA.id
submission['label']=predictions_A


Predicting: 100%|██████████| 3126/3126 [00:21<00:00, 145.62it/s]


In [13]:
submission.to_csv("mySubmission.csv")